In [227]:
import folium
import pandas as pd
# 설치 안되어 있을 시 cmd 에서 pip install folium
from folium.plugins import MarkerCluster
import json
from folium import plugins
import os
print(folium.__version__)
import numpy as np

0.10.1


In [244]:
coronaItaly = pd.read_csv('./corona.csv')

In [245]:
coronaItaly

,Date,Piedmont,Liguria,Lombardy,Veneto,Trentino-Alto Adige,Emilia-Romaña,Toscana,Lazio,Sicily
0,2020-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN
1,2020-02-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0,NaN
2,2020-02-21,NaN,NaN,15.0,2.0,NaN,NaN,NaN,NaN,NaN
3,2020-02-22,1.0,NaN,40.0,16.0,NaN,2.0,NaN,NaN,NaN
4,2020-02-23,2.0,NaN,57.0,7.0,NaN,7.0,NaN,NaN,NaN
5,2020-02-24,NaN,NaN,61.0,8.0,1.0,9.0,NaN,NaN,NaN
6,2020-02-25,NaN,2.0,68.0,13.0,NaN,8.0,2.0,NaN,3.0


In [246]:
coronaItaly = coronaItaly.fillna(0)

In [247]:
coronaItalyT=coronaItaly.T



In [248]:
coronaItalyT.columns=coronaItalyT.loc['Date',:].tolist()

In [249]:
coronaItalyT

,2020-01-31,2020-02-06,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25
Date,2020-01-31,2020-02-06,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25
Piedmont,0,0,0,1,2,0,0
Liguria,0,0,0,0,0,0,2
Lombardy,0,0,15,40,57,61,68
Veneto,0,0,2,16,7,8,13
Trentino-Alto Adige,0,0,0,0,0,1,0
Emilia-Romaña,0,0,0,2,7,9,8
Toscana,0,0,0,0,0,0,2
Lazio,2,1,0,0,0,0,0
Sicily,0,0,0,0,0,0,3


In [250]:
accum=coronaItalyT.iloc[1:,:]
accum

,2020-01-31,2020-02-06,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25
Piedmont,0,0,0,1,2,0,0
Liguria,0,0,0,0,0,0,2
Lombardy,0,0,15,40,57,61,68
Veneto,0,0,2,16,7,8,13
Trentino-Alto Adige,0,0,0,0,0,1,0
Emilia-Romaña,0,0,0,2,7,9,8
Toscana,0,0,0,0,0,0,2
Lazio,2,1,0,0,0,0,0
Sicily,0,0,0,0,0,0,3


In [251]:
accum = np.cumsum(accum, axis=1)
accum

,2020-01-31,2020-02-06,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25
Piedmont,0,0,0,1,3,3,3
Liguria,0,0,0,0,0,0,2
Lombardy,0,0,15,55,112,173,241
Veneto,0,0,2,18,25,33,46
Trentino-Alto Adige,0,0,0,0,0,1,1
Emilia-Romaña,0,0,0,2,9,18,26
Toscana,0,0,0,0,0,0,2
Lazio,2,3,3,3,3,3,3
Sicily,0,0,0,0,0,0,3


In [252]:
accum['2020-02-23']

Piedmont                 3
Liguria                  0
Lombardy               112
Veneto                  25
Trentino-Alto Adige      0
Emilia-Romaña            9
Toscana                  0
Lazio                    3
Sicily                   0
Name: 2020-02-23, dtype: object

In [254]:
chrol=accum.reset_index()
chrol

,index,2020-01-31,2020-02-06,2020-02-21,2020-02-22,2020-02-23,2020-02-24,2020-02-25
0,Piedmont,0,0,0,1,3,3,3
1,Liguria,0,0,0,0,0,0,2
2,Lombardy,0,0,15,55,112,173,241
3,Veneto,0,0,2,18,25,33,46
4,Trentino-Alto Adige,0,0,0,0,0,1,1
5,Emilia-Romaña,0,0,0,2,9,18,26
6,Toscana,0,0,0,0,0,0,2
7,Lazio,2,3,3,3,3,3,3
8,Sicily,0,0,0,0,0,0,3


In [259]:
## topojson to geojson

#http://bl.ocks.org/enjalot/63d06e2ccadad0cb30dc5f920efd1cdf

import pandas as pd
state_geo = "./italy/italy-regions1.json"

m = folium.Map(location = [41.702403, 11.394294 ], zoom_start = 6.3,tiles = 'Cartodb Positron')

folium.Choropleth(
    geo_data=state_geo,
     name='choropleth',
    data=chrol,
    data_out='data1.json',
    columns=['index', '2020-02-25'],
    key_on='feature.properties.NAME_1',
    fill_color='OrRd',
    fill_opacity=0.7,
    line_opacity=0.2, 
    nan_fill_color='Green',
    nan_fill_opacity=0.1
#     topojson='objects.ITA_adm1'
#     legend_name='Unemployment Rate (%)'
).add_to(m)

folium.LayerControl().add_to(m)

m

In [224]:
## topojson으로 - 미완성

#https://stackoverflow.com/questions/57365733/is-my-topojson-file-structured-properly-in-order-to-render-a-map-on-folium
#https://gist.github.com/wrobstory/5609889#file-us_counties_20m_topo-json

#참고
import branca


colorscale = branca.colormap.linear.YlOrRd_09.scale(0, 50e3)
employed_series = accum['23-02-2020']


def style_function(feature):
    employed = employed_series.get(str(feature['properties']['NAME_1']), None)
#     employed = employed_series.get(str(feature['NAME_1']), None)
    return {
        'fillOpacity': 0.5,
        'weight': 0,
        'fillColor': '#black' if employed is None else colorscale(employed)
        }


m = folium.Map(
    location=[41.702403, 11.394294 ],
    tiles='cartodbpositron',
    zoom_start=6
)

folium.TopoJson(
     json.load(open('./italy/italy-regions.json')),
    'objects.ITA_adm1',
    style_function=style_function
).add_to(m)


m

In [256]:
help(folium.TopoJson)

Help on class TopoJson in module folium.features:

class TopoJson(folium.map.Layer)
 |  TopoJson(data, object_path, style_function=None, name=None, overlay=True, control=True, show=True, smooth_factor=None, tooltip=None)
 |  
 |  Creates a TopoJson object for plotting into a Map.
 |  
 |  Parameters
 |  ----------
 |  data: file, dict or str.
 |      The TopoJSON data you want to plot.
 |      * If file, then data will be read in the file and fully
 |      embedded in Leaflet's JavaScript.
 |      * If dict, then data will be converted to JSON and embedded
 |      in the JavaScript.
 |      * If str, then data will be passed to the JavaScript as-is.
 |  object_path: str
 |      The path of the desired object into the TopoJson structure.
 |      Ex: 'objects.myobject'.
 |  style_function: function, default None
 |      A function mapping a TopoJson geometry to a style dict.
 |  name : string, default None
 |      The name of the Layer, as it will appear in LayerControls
 |  overlay : bo

m = folium.Map(location = [41.702403, 11.394294 ], zoom_start = 6.3,tiles = 'Cartodb Positron')

folium.TopoJson(
    json.loads("./italy/italy-regions.json"),
    'objects.ITA_adm1',
    name='topojson'
).add_to(m)

folium.LayerControl().add_to(m)


m

map_3 = folium.Map(location=[40, -99], zoom_start=4)
map_3.geo_json(geo_path=county_geo, data_out='data3.json', data=df,
               columns=['GEO_ID', 'Median_Household_Income_2011'],
               key_on='feature.id',
               fill_color='PuRd', line_opacity=0.3,
               legend_name='Median Household Income 2011 ($)',
               topojson='objects.us_counties_20m')
map_3.create_map(path='map_3.html')